In [1]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from datetime import timedelta
import pytz
import re
import fuzzywuzzy
from fuzzywuzzy import process
from scipy import stats

c:\Users\Karlo\AppData\Local\Programs\Python\Python310\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
# Import csv file and use first row as header
df = pd.read_csv("jefit.csv", header=1)

In [3]:
# Print out the first five rows of dataframe
df.head()

,row_id,USERID,TIMESTAMP,gender,currentRoutine,DOB,name,length,mass,dbversion,...,use_location,screenon,auto_lock,lastlogs,preloadreps,exp_level,fit_goal,audio_reminder,audio_exercise_tips,audio_personal_tips
0,40172618,7317810,2022-07-04 14:29:07,M,29,1998-11-29,NaN,cm,kg,NaN,...,0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1,##############################################...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,### PROFILE ##################################...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,row_id,USERID,TIMESTAMP,_id,mydate,weight,fatpercent,chest,arms,waist,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,44341730,7317810,2019-04-27 18:40:26,1,2019-02-11,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Find every row that contains multiple "#" characters using regex
mask = df.row_id.str.contains(r"#+")
df[mask]

,row_id,USERID,TIMESTAMP,gender,currentRoutine,DOB,name,length,mass,dbversion,...,use_location,screenon,auto_lock,lastlogs,preloadreps,exp_level,fit_goal,audio_reminder,audio_exercise_tips,audio_personal_tips
1,##############################################...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,### PROFILE ##################################...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,##############################################...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,### ROUTINES #################################...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
751,##############################################...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
752,### WORKOUT SESSIONS #########################...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1298,##############################################...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1299,### EXERCISE LOGS ############################...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5080,##############################################...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5081,### EXERCISE RECORDS #########################...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
sessions = df.iloc[752:1299].reset_index(drop=True)
exercises = df.iloc[1299:5081].reset_index(drop=True)

In [6]:
#Save files as csv for further cleaning
sessions.to_csv("sessions.csv")
exercises.to_csv("exercises.csv")